In [ ]:
import numpy as np
import pandas as pd
from scipy.fftpack import fft, ifft, fftfreq
import scipy.signal as signal
import matplotlib.pyplot as plt
from configparser import ConfigParser

In [ ]:
folder = 'Single transducer extended response'
#folder = 'New board/' + folder
folder = '../Measurements/' + folder + '/'
config = ConfigParser()
config.read(folder + 'settings.ini')

timedata=pd.read_csv(folder + 'F1.txt',delimiter='\t')
n=len(timedata)
fs=float(config['General']['fs'])

calibration = float(config['General']['Microphone calibration'])#3.55e-3 # mV/P
#calibration = 10**((-49.4-.4)/20)
dB = lambda x: 20*np.log10(np.abs(x))
dBuSPL = lambda x: 20*np.log10(np.abs(x)/(20e-6*calibration))
dBu2SPL = lambda x: 10*np.log10(np.abs(x)/(20e-6*calibration)**2)

spectrogram=timedata.apply(lambda x: signal.spectrogram(x,fs=fs,nperseg=2**14,scaling='spectrum'))
win = signal.windows.flattop(n)
win = signal.windows.boxcar(n)
frequencydata=timedata.apply(lambda x:fft(x*win),raw=True)/win.sum()
f=fftfreq(n,1/fs)

In [ ]:
channel='Microphone'
plotType = 'Response' # 'Spectrogram' 'Response'
scale = 'SPL' # 'SPL' 'dB' 'V' 'Pa' 
fc = 200e3
fd = 200
fMin = fc-fd
fMax = fc+fd
fMin = 20e3
fMax = 500e3
dBMax = None
dBMin = 20

tMin = 0
tMax = 100

if plotType == 'Spectrogram':
    if scale == 'SPL':
        fun = dBu2SPL
    elif scale == 'dB':
        fun = dB
    elif scale == 'Pa' :
        fun = lambda x: x/20e-6/calibration
    else:
        fun = lambda x: x    
    tIdx = (spectrogram[channel][1]>tMin) & (spectrogram[channel][1]<tMax)
    fIdx = (spectrogram[channel][0]>fMin) & (spectrogram[channel][0]<fMax) 
    plt.pcolormesh(spectrogram[channel][1][tIdx],
                   spectrogram[channel][0][fIdx],
                   fun(spectrogram[channel][2][np.ix_(fIdx,tIdx)]),
                   vmin=dBMin,vmax=dBMax)              
#    plt.ylim((fMin,fMax))
    plt.colorbar()
    plt.show()
else:
    if scale == 'SPL':
        fun = dBuSPL
    elif scale == 'dB':
        fun = dB
    elif scale == 'Pa' :
        fun = lambda x: np.abs(x)/20e-6/calibration
    else:
        fun = lambda x: np.abs(x)   
    fIdx = (f>fMin) & (f<fMax)
    plt.plot(f[fIdx],fun(frequencydata[channel][fIdx]))
    plt.ylim(ymin=dBMin,ymax=dBMax)
    plt.show()
    maxVal = np.max(fun(frequencydata[channel][fIdx]))
    print('Max value in "{}": {:.1f} {}'.format(
        channel, maxVal, scale
        ))    

In [ ]:
channel_1 = 'Microphone'
channel_2 = 'Amplifier'
fStart = float(config['Signal']['frequency'])
fEnd = float(config['Signal']['parameter 2'])

fMin = fStart
fMax = fEnd

#fMin = 60
#fMax = 4e3

dbMin = -60

response = frequencydata[channel_1]/frequencydata[channel_2]
phase = np.unwrap(np.angle(response))
idx = (f>fStart) & (f<fEnd)
phase -= np.max(phase[idx])
response /= np.max(np.abs(response[idx]))
idx = (f>fMin) & (f<fMax)
plt.plot(f[idx],dB(response[idx]))
plt.ylim(ymin=dbMin)
plt.show()

plt.plot(f[idx],phase[idx]/np.pi)
plt.show()
